In [31]:
# %load ../import_data.py
from collections import Counter
import pandas as pd
import numpy as np
pd.set_option('display.max_colwidth', 1000)
pd.set_option('display.max_rows', 20)

import xgboost as xgb

import os,sys,inspect
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
sys.path.insert(0,parentdir)

from statistic import Statistic
from utils import UtilsKy
from analyzer import HelperAnalyzer, AnalyzerPrediction

# for autoreload modules
%load_ext autoreload
%autoreload 2

# kyw3
path_data = '/mnt/files/workdata/work/merchants/merchant_33_kyw3_2020-06-05/04_experiments/ex_01_some_teach/'
db_teach = pd.read_csv(UtilsKy.DB_TEACH_KYW3, dtype=str, encoding='cp1251')
db_test = pd.read_csv(UtilsKy.DB_TEST_KYW3, dtype=str, encoding='cp1251')
white = pd.read_csv(UtilsKy.WHITE_KYW3 , dtype=str)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [32]:
# %load ../prepare_data.py
COL_FACTORS = ['bin', 'amount', 'bank_currency', 'hour', 'day_of_week', 'longitude', 'latitude', 'phone_2_norm', 'is_gender_undefined', 'is_city_resolved']
COL_FACTORS = sorted(COL_FACTORS)

# For Xgboost
from helper import DataHelper
datahelper = DataHelper(db_teach, db_test, COL_FACTORS)
datahelper.create_train_test()
datahelper.show_columns_with_na()
mean_values = datahelper.get_mean_value()
replaced_values = { col: mean_values[col] for col in ('latitude', 'longitude')}
replaced_values['default'] =  -999
datahelper.replaced_na_values(replaced_values)   
train , test = datahelper.get_train_test()

Statistic na values in columns : 
train na columns : Index(['latitude', 'longitude'], dtype='object')
latitude_na: 1537,longitude_na: 1537
test na columns : Index(['latitude', 'longitude'], dtype='object')
latitude_na: 63,longitude_na: 63
Replaced na values:
is_gender_undefined_na -> -999
is_city_resolved_na -> -999
bin_na -> -999
hour_na -> -999
day_of_week_na -> -999
bank_currency_na -> -999
phone_2_norm_na -> -999
amount_na -> -999
longitude_na -> -92.53325861542274
latitude_na -> 36.90237577890762


In [34]:
from sklearn.linear_model import LinearRegression

In [44]:
for factor in COL_FACTORS:
    other_factors = [col for col in COL_FACTORS if col!= factor]
    X, y = train[other_factors].values, train[factor].values
    clf = LinearRegression()
    clf.fit(X, y)

    print("{} : {}" .format(factor, clf.score(X, y)))    

amount : 0.010485544310996242
bank_currency : 0.08606761763973025
bin : 0.015339974054812244
day_of_week : 0.011961003882192345
hour : 0.013473552284439805
is_city_resolved : 0.03287643302635179
is_gender_undefined : 0.002518759050812336
latitude : 0.11289763580537071
longitude : 0.126593140919532
phone_2_norm : 0.007217760706448306
status : 0.0023797886304331994
